In [4]:
# Step 1: Install Required Libraries
!pip install gradio scikit-learn langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=83934b560af4c8323fca79f29f4fcc599b436e96e1294460158eb824d6b2a10a
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [5]:
# Step 2: Create Enhanced Dataset
sample_data = """
symptoms,disease
"fever, chills, headache, vomiting, sweating",malaria
"persistent cough, chest pain, fatigue, weight loss, night sweats",tuberculosis
"diarrhea, dehydration, nausea, stomach cramps, vomiting",cholera
"runny nose, sore throat, cough, sneezing, body aches",flu
"high fever, joint pain, headache, muscle pain, rash",dengue
"weight loss, night sweats, prolonged fever, swollen lymph nodes",hiv
"kupoteza uzito, kutokwa jasho usiku, homa ya muda mrefu, tezi kuvimba",hiv
"kuharisha, kichefuchefu, maumivu ya tumbo, upungufu wa maji mwilini",cholera
"maumivu ya kichwa, homa kali, maumivu ya viungo, vipele",dengue
"kutokwa na makamasi, kikohozi, koo kuuma, mafua",flu
"""

with open("kenya_diseases_swahili.csv", "w") as f:
    f.write(sample_data.strip())

In [6]:
# Step 3: Load & Train Model
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from langdetect import detect

df = pd.read_csv("kenya_diseases_swahili.csv")

X = df["symptoms"]
y = df["disease"]

model = make_pipeline(TfidfVectorizer(), LogisticRegression())
model.fit(X, y)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression())])

In [7]:
# Step 4: Define Processing Function
def diagnose(symptom_input):
    if not symptom_input or len(symptom_input.split()) < 2:
        return "⚠️ Tafadhali andika au sema dalili halisi. / Please try again with valid symptoms."

    try:
        lang = detect(symptom_input)
    except:
        lang = 'en'

    prediction = model.predict([symptom_input])[0]
    return f"✅ Ugonjwa Unaowezekana / Likely Disease: **{prediction.upper()}**\n\n📌 Ushauri: Wasiliana na daktari kwa ushauri zaidi."


In [8]:
# Step 5: Define Gradio Interface with Voice & Validation
import gradio as gr

iface = gr.Interface(
    fn=diagnose,
    inputs=[
        gr.Textbox(lines=2, placeholder="Andika dalili hapa... / Type symptoms here"),
        gr.Audio("microphone", label="🎤 Rekodi sauti yako / Record your symptoms (English or Kiswahili)")  # Removed 'source=' and made "microphone" a positional argument
    ],
    outputs="markdown",
    title="🩺 Chatbot ya Ugonjwa Kenya - Kenya Disease Chatbot",
    description="⚕️ Andika au sema dalili zako kwa Kiswahili au Kiingereza. Chatbot itakupa utambuzi wa ugonjwa unaowezekana.",
    live=True
)

/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1018: UserWarning: Expected 1 arguments for function <function diagnose at 0x7c4b0a8449a0>, received 2.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1026: UserWarning: Expected maximum 1 arguments for function <function diagnose at 0x7c4b0a8449a0>, received 2.
  warnings.warn(


In [9]:
# Optional: Whisper-like transcribe audio (simulation with gradio's audio input)
!pip install SpeechRecognition

def process_input(text_input, audio_path):
    if audio_path:
        recognizer = sr.Recognizer()
        with sr.AudioFile(audio_path) as source:
            audio_data = recognizer.record(source)
        try:
            transcribed = recognizer.recognize_google(audio_data)
            return diagnose(transcribed)
        except:
            return "⚠️ Hatukuweza kuelewa sauti yako. / Unable to process your voice input."

    return diagnose(text_input)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 40.7 MB/s eta 0:00:00


In [10]:
# Final interface setup
iface = gr.Interface(
    fn=process_input,
    inputs=[
        gr.Textbox(label="📝 Dalili / Symptoms"),
        gr.Audio(type="filepath", label="🎤 Rekodi / Record Voice")  # Removed 'source="microphone"'
    ],
    outputs="markdown",
    title="🇰🇪 Kenya Disease Diagnosis Chatbot (Swahili + English)",
    description="🚑 Andika au sema dalili zako kama: 'kichwa kuuma, kutapika, homa' au 'fever, cough'.",
    theme="default"
)

In [11]:
# Launch app
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://357c7713c7734b9521.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
